In [ ]:
from torch import nn
from common import summary, load_data_from_mnist, train


In [ ]:
def vgg_block(num_convs: int, in_channels: int, out_channels: int):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)


In [ ]:
def vgg(conv_arch: list):  # VGG11
    conv_blocks = []
    in_channels = 1

    for num_convs, out_channels in conv_arch:
        conv_blocks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels  # front out channel = end in channel

    return nn.Sequential(
        *conv_blocks,
        nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096),  # 224 / (2**5) = 7
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )


In [ ]:
conv_arch = [(1, 64), (1, 128), (2, 256), (2, 512), (2, 512)]
summary(vgg(conv_arch), (1, 1, 224, 224))


In [ ]:
ratio = 4
small_conv_arch = [(x[0], x[1] // ratio) for x in conv_arch]
net = vgg(small_conv_arch)
summary(net, (1, 1, 224, 224))


In [ ]:
train(net, *load_data_from_mnist(resize=224), 1, 0.05, "ch6")
